In [1]:
from pprint import pprint
from transformers import AutoModelForObjectDetection, AutoFeatureExtractor
import torchvision
from pathlib import Path
import torch

/root/.cache/pypoetry/virtualenvs/leaf-area-MmBPJx5b-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from roboflow import Roboflow
rf = Roboflow(api_key="onQHDhSGeUEnGY4e1Qq7")
project = rf.workspace("leafframes").project("leaf-area-zxdye")
version = project.version(3)
dataset = version.download("coco")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to leaf-area-3 in coco:: 100%|██████████| 107/107 [00:00<00:00, 382.42it/s]


In [5]:
MODEL_NAME = 'facebook/detr-resnet-50'

model = AutoModelForObjectDetection.from_pretrained(MODEL_NAME)
feature_extractor = AutoFeatureExtractor.from_pretrained(MODEL_NAME)

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
# feature_extractor, 
model

DetrForObjectDetection(
  (model): DetrModel(
    (backbone): DetrConvModel(
      (conv_encoder): DetrConvEncoder(
        (model): FeatureListNet(
          (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (bn1): DetrFrozenBatchNorm2d()
          (act1): ReLU(inplace=True)
          (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
          (layer1): Sequential(
            (0): Bottleneck(
              (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn1): DetrFrozenBatchNorm2d()
              (act1): ReLU(inplace=True)
              (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn2): DetrFrozenBatchNorm2d()
              (drop_block): Identity()
              (act2): ReLU(inplace=True)
              (aa): Identity()
              (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

In [7]:
class LeafDetectionCocoDataset(torchvision.datasets.CocoDetection):
    def __init__(self, img_folder, feature_extractor):
        ann_file = img_folder / '_annotations.coco.json'
        super(LeafDetectionCocoDataset, self).__init__(root=img_folder, annFile=ann_file)
        self.feature_extractor = feature_extractor

    def __getitem__(self, idx):
        img, targets = super(LeafDetectionCocoDataset, self).__getitem__(idx)
        image_id = self.ids[idx]
        annotations = {'image_id': image_id, 'annotations': targets}
        encoding = self.feature_extractor(images=img, annotations=annotations, return_tensor='pt')
        pixel_values = encoding['pixel_values'][0]
        target = encoding['labels'][0]
        return pixel_values, target

In [9]:
dataset = LeafDetectionCocoDataset(img_folder=Path('/home/piotr/projects/leaf-area/leaf-area/notebooks/leaf-area-3/train'),
                                   feature_extractor=feature_extractor)

loading annotations into memory...
Done (t=0.15s)
creating index...
index created!


In [10]:
pixel_values, target = dataset[0]
print(pixel_values.shape)
pprint(target)

Unused or unrecognized kwargs: return_tensor.
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


(3, 800, 1066)
{'area': array([62378.344, 47712.457, 69989.73 , 27355.693, 37359.75 , 40416.895,
       39824.625], dtype=float32),
 'boxes': array([[0.3765    , 0.33333337, 0.23050001, 0.31733334],
       [0.5955    , 0.6460001 , 0.21299998, 0.26266667],
       [0.46824998, 0.42650002, 0.24449997, 0.3356667 ],
       [0.320875  , 0.22900002, 0.15275002, 0.21      ],
       [0.66175   , 0.33850002, 0.17499998, 0.25033334],
       [0.67649996, 0.73316675, 0.19450003, 0.24366672],
       [0.39299998, 0.695     , 0.199     , 0.23466668]], dtype=float32),
 'class_labels': array([1, 1, 1, 1, 1, 1, 1]),
 'image_id': array([0]),
 'iscrowd': array([0, 0, 0, 0, 0, 0, 0]),
 'orig_size': array([3000, 4000]),
 'size': (800, 1066)}


In [11]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    pixel_values = [item[0] for item in batch]
    encoding = feature_extractor.pad(pixel_values, return_tensors='pt')
    labels = [item[1] for item in batch]
    return {
        'pixel_values': encoding['pixel_values'],
        'labels': labels
    }

In [12]:
dataloader = DataLoader(dataset, collate_fn=collate_fn, batch_size = 2)
batch = next(iter(dataloader))

Unused or unrecognized kwargs: return_tensor.
Unused or unrecognized kwargs: return_tensor.


In [13]:
print(batch['pixel_values'].shape)
pprint(batch['labels'])

torch.Size([2, 3, 800, 1066])
[{'area': array([62378.344, 47712.457, 69989.73 , 27355.693, 37359.75 , 40416.895,
       39824.625], dtype=float32),
  'boxes': array([[0.3765    , 0.33333337, 0.23050001, 0.31733334],
       [0.5955    , 0.6460001 , 0.21299998, 0.26266667],
       [0.46824998, 0.42650002, 0.24449997, 0.3356667 ],
       [0.320875  , 0.22900002, 0.15275002, 0.21      ],
       [0.66175   , 0.33850002, 0.17499998, 0.25033334],
       [0.67649996, 0.73316675, 0.19450003, 0.24366672],
       [0.39299998, 0.695     , 0.199     , 0.23466668]], dtype=float32),
  'class_labels': array([1, 1, 1, 1, 1, 1, 1]),
  'image_id': array([0]),
  'iscrowd': array([0, 0, 0, 0, 0, 0, 0]),
  'orig_size': array([3000, 4000]),
  'size': (800, 1066)},
 {'area': array([ 9466.791 ,  8350.618 ,  8558.417 ,  9909.75  ,  7978.9395,
        7522.407 ,  9548.162 ,  8065.925 ,  9048.493 ,  7934.2383,
        7813.9224, 14427.458 ,  7342.6084,  9274.2   ,  9757.596 ,
        5369.6553,  8741.84  , 11261.

In [14]:
import pytorch_lightning as pl
import numpy as np

In [15]:
from pytorch_lightning.utilities.types import TRAIN_DATALOADERS

class LeafAreaDetrDataModule(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
        self.feature_extractor = AutoFeatureExtractor.from_pretrained(MODEL_NAME)

    def collate_fn(self, batch):
        pixel_values = [item[0] for item in batch]
        encoding = self.feature_extractor.pad(pixel_values, return_tensors='pt')
        labels = [item[1] for item in batch]
        batch = {}
        batch['pixel_values'] = encoding['pixel_values']
        batch['labels'] = labels
        return batch
    
    def train_dataloader(self):
        train_dataset = LeafDetectionCocoDataset(img_folder=Path('leaf-area-3/train'),
                                                 feature_extractor=self.feature_extractor)
        return DataLoader(train_dataset, collate_fn=self.collate_fn, batch_size=2, shuffle=True)

    def val_dataloader(self):
        valid_dataset = LeafDetectionCocoDataset(img_folder=Path('leaf-area-3/valid'),
                                                 feature_extractor=self.feature_extractor)
        return DataLoader(valid_dataset, collate_fn=self.collate_fn, batch_size=2, shuffle=False)

    def test_dataloader(self):
        test_dataset = LeafDetectionCocoDataset(img_folder=Path('leaf-area-3/test'),
                                                 feature_extractor=self.feature_extractor)
        return DataLoader(test_dataset, collate_fn=self.collate_fn, batch_size=2, shuffle=False)
    
    @property
    def num_labels(self):
        train_dataset = LeafDetectionCocoDataset(img_folder=Path('leaf-area-3/train'),
                                                 feature_extractor=self.feature_extractor)
        # return np.unique(list(train_dataset.coco.cats.keys()))
        return train_dataset.coco.cats.keys()
        
        

In [16]:
datamodule = LeafAreaDetrDataModule()
datamodule.num_labels

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!


The class DetrFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use DetrImageProcessor instead.


dict_keys([0, 1])

In [17]:
class Detr(pl.LightningModule):
    def __init__(self, 
                 num_labels,
                 lr, weight_decay):
        super().__init__()
        self.model = AutoModelForObjectDetection.from_pretrained(
            MODEL_NAME,
            num_labels=num_labels,
            ignore_mismached_sizes=True
        )
        self.lr = lr
        self.weight_decay = weight_decay

    def forward(self, pixel_values):
        outputs = self.model(pixel_values=pixel_values)
        return outputs
    
    def common_step(self, batch):
        pixel_values = batch['pixel_values']
        labels = [{k: v.to(self.device) for k, v in target.items()} for target in batch['labels']]
        outputs = self.model(pixel_values=pixel_values, labels=labels)
        loss = outputs.loss
        loss_dict = outputs.loss_dict
        return loss, loss_dict
    
    def training_step(self, batch):
        loss, loss_dict = self.common_step(batch)
        self.log('training_loss', loss)
        for k, v in loss_dict.items():
            self.log('train_' + k, v.item())
        return loss
    
    def validation_step(self, batch):
        loss, loss_dict = self.common_step(batch)
        self.log('validation_loss', loss)
        for k, v in loss_dict.items():
            self.log('validation_' + k, v.item())
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.paramters(),
                                      lr=self.lr,
                                      weight_decay=self.weight_decay)
        return optimizer

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint, Callback

model = Detr(num_labels=datamodule.num_labels,
             lr=2.5e-5,
             weight_decay=1e-4)

trainer = Trainer(max_steps=20000,
                  gradient_clip_val=0.1,
                  accumulate_grad_batches=4,
                  callbacks=[
                      EarlyStopping(monitor='validation_loss', patience=3, mode='min'),
                      ModelCheckpoint(monitor='validation_loss', mode='min', save_top_k=1, dirpath='checkpoints'),
                  ])

trainer.fit(model, datamodule)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


TypeError: cannot pickle 'dict_keys' object